In [1]:
import requests
import urllib3

In [2]:
requests.packages.urllib3.disable_warnings()
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS += ':HIGH:!DH:!aNULL'
try:
    requests.packages.urllib3.contrib.pyopenssl.util.ssl_.DEFAULT_CIPHERS += ':HIGH:!DH:!aNULL'
except AttributeError:
    # no pyopenssl support used / needed / available
    pass

In [3]:
url = 'https://www.officialcharts.com/charts/'

In [4]:
#html = requests.get(url, verify=False)
html = requests.get(url)

In [5]:
html.content

b'<!DOCTYPE html>\n<html lang="en">\n<head><meta charset="utf-8">\n<title>The Official Charts | Official Charts</title>\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<meta name="msapplication-TileColor" content="#f5f2e7">\n<meta name="msapplication-config" content="/favicons/browserconfig.xml">\n<meta name="theme-color" content="#f5f2e7">\n<link rel="apple-touch-icon" sizes="180x180" href="/favicons/apple-touch-icon.png">\n<link rel="icon" type="image/png" sizes="32x32" href="/favicons/favicon-32x32.png">\n<link rel="icon" type="image/png" sizes="16x16" href="/favicons/favicon-16x16.png">\n<link rel="manifest" href="/favicons/site.webmanifest">\n<link rel="mask-icon" color="#0029f3" href="favicons/safari-pinned-tab.svg">\n<link rel="shortcut icon" href="/favicons/favicon.ico">\n<script src="/js/quantcast.js" async="async"></script>\n<script src="https://cdn.tagdeliver.com/cipt/17187.js" async="async"></script>\n<meta name="title" content="The Official Charts | 

In [6]:
from bs4 import BeautifulSoup

In [7]:
soup = BeautifulSoup(html.content)

In [8]:
soup

<!DOCTYPE html>
<html lang="en">
<head><meta charset="utf-8"/>
<title>The Official Charts | Official Charts</title>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="#f5f2e7" name="msapplication-TileColor"/>
<meta content="/favicons/browserconfig.xml" name="msapplication-config"/>
<meta content="#f5f2e7" name="theme-color"/>
<link href="/favicons/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicons/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicons/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/favicons/site.webmanifest" rel="manifest"/>
<link color="#0029f3" href="favicons/safari-pinned-tab.svg" rel="mask-icon"/>
<link href="/favicons/favicon.ico" rel="shortcut icon"/>
<script async="async" src="/js/quantcast.js"></script>
<script async="async" src="https://cdn.tagdeliver.com/cipt/17187.js"></script>
<meta content="The Official Charts | Official Charts" nam

## Localizando la información

### Nombre columnas

In [9]:
columnas = [x.text for x in soup.find_all('tr')]
columnas

[]

In [10]:
columnas = ['pos', 'artist', 'title']
columnas

['pos', 'artist', 'title']

### Posición

In [11]:
pos = [x.text for x in soup.find_all('span', {'class': 'position'})]

### Artist

In [12]:
artist = [x.text.replace('\n', '').title() for x in soup.find_all('div', {'class': 'artist'})]

### Titulo

In [13]:
titulos = [x.text.replace('\n', '').title() for x in soup.find_all('div', {'class': 'title'})]

### Label

In [14]:
label = [x.text.replace('\n', '').title() for x in soup.find_all('span', {'class': 'label'})]

In [15]:
len(artist)

0

### Table Rows

In [16]:
columns_tr = [x.text for x in soup.find_all('tr')][0].split('\n')

IndexError: list index out of range

In [ ]:
data = [x.text for x in soup.find_all('tr')][1:]

In [ ]:
data[0:5]

['\n\n1\n\n\n\r\n                    2\r\n                \n\n\n\n\n\n\n\n\nMIRACLE\n\n\nCALVIN HARRIS/ELLIE GOULDING\n\nCOLUMBIA\n\n\n\n\n1\n\n7\n\n\nBuy\nListen\n\n\n\n\n\n',
 '\n\n\nBuy\nListen\n\n\n',
 '\n\n\namazon\nitunes\n\n\n\n',
 '\n\n\n\n\n\n',
 '\n\n\ndeezer\nspotify\n\n\n\n']

In [ ]:
def limpiar_filas(lista):
    new_list = []
    for row in data:
        try:
            int(row[2])
            new_row = list(filter(None, row.split('\n')))
            new_list.append(new_row)
        except:
            pass
    return new_list

In [ ]:
rows = limpiar_filas(data)

In [ ]:
rows[0]

['1',
 '\r',
 '                    2\r',
 '                ',
 'MIRACLE',
 'CALVIN HARRIS/ELLIE GOULDING',
 'COLUMBIA',
 '1',
 '7',
 'Buy',
 'Listen']

In [ ]:
pos = [row[0] for row in rows]
LW = [row[2] for row in rows]
title = [row[4].title() for row in rows]
artist = [row[5].title() for row in rows]
label = [row[6].title() for row in rows]

## Construyendo un Web Spider

In [ ]:
import pandas as pd

In [ ]:
dct = {
    'pos': pos,
    'artist': artist,
    'title': title,
    'label': label
}

In [ ]:
import datetime

In [ ]:
df = pd.DataFrame(dct)

df['chart'] = 'UK'
semana = datetime.datetime.today().strftime('%V')
df['week'] = int(semana)

In [ ]:
df.head()

,pos,artist,title,label,chart,week
0,1,Calvin Harris/Ellie Goulding,Miracle,Columbia,UK,17
1,2,David Kushner,Daylight,Miserable Music,UK,17
2,3,Lewis Capaldi,Wish You The Best,Vertigo,UK,17
3,4,Libianca,People,5K,UK,17
4,5,Rema,Calm Down,Mavin,UK,17


## Construyendo spider

In [ ]:
def listas_uk(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content)
    columnas = ['pos', 'artist', 'title', 'label']
    pos = [x.text for x in soup.find_all('span', {'class': 'position'})]
    artist = [x.text.replace('\n', '').title() for x in soup.find_all('div', {'class': 'artist'})]
    title = [x.text.replace('\n', '').title() for x in soup.find_all('div', {'class': 'title'})]
    label = [x.text.replace('\n', '').title() for x in soup.find_all('span', {'class': 'label'})]
    dct = {
        'pos': pos,
        'artist': artist,
        'title': title,
        'label': label
        }
    df = pd.DataFrame(dct)
    df['chart'] = 'UK'
    semana = datetime.datetime.today().strftime('%V')
    df['week'] = int(semana)
    return df

In [ ]:
nuevos_datos = listas_uk(url)

In [ ]:
nuevos_datos['artist'].value_counts()

Lewis Capaldi          3
Ed Sheeran             3
Taylor Swift           3
Luke Combs             2
Strandz                2
                      ..
Ayra Starr             1
Jack Black             1
Niall Horan            1
Zara Larsson           1
Davido Ft Musa Keys    1
Name: artist, Length: 89, dtype: int64

## Acceso URLS

In [ ]:
url = 'https://www.officialcharts.com/charts/singles-chart/20230421/7501/'
fecha = '20230421'

In [ ]:
fecha_obj = datetime.datetime.strptime(fecha, '%Y%m%d')

In [ ]:
fecha_nueva = fecha_obj - datetime.timedelta(days=7)

In [ ]:
fecha_nueva.strftime('%Y%m%d')

'20230414'

In [ ]:
def restar_semanas(fecha_str, semanas):
    fecha_obj = datetime.datetime.strptime(fecha_str, '%Y%m%d')
    fecha_nueva = fecha_obj - datetime.timedelta(weeks=semanas)
    return fecha_nueva.strftime('%Y%m%d')

In [ ]:
fecha

'20230421'

In [ ]:
restar_semanas(fecha, semanas=3)

'20230331'

### Cambiar URL

In [ ]:
url

'https://www.officialcharts.com/charts/singles-chart/20230421/7501/'

In [ ]:
fecha = '20220909'
url_nueva = 'https://www.officialcharts.com/charts/singles-chart/' + fecha + '/7501/'

### Construir Spider múltiple

In [ ]:
def spider_listas(fecha, semanas):
    df_list = []
    for i in range(semanas):
        nueva_fecha = restar_semanas(fecha, i)
        url_nueva = 'https://www.officialcharts.com/charts/singles-chart/' + nueva_fecha + '/7501/'
        df = listas_uk(url_nueva)
        semana = datetime.datetime.strptime(nueva_fecha, '%Y%m%d').strftime('%V')
        df['week'] = semana
        df_list.append(df)
    df= pd.concat(df_list)
    return df

In [ ]:
df= spider_listas(fecha, 10)

In [ ]:
df

,pos,artist,title,label,chart,week
0,1,Sam Smith & Kim Petras,Unholy,Emi,UK,39
1,2,David Guetta Ft Bebe Rexha,I'M Good (Blue),Parlophone,UK,39
2,3,Lewis Capaldi,Forget Me,Vertigo,UK,39
3,4,Eliza Rose/Interplanetary,B.O.T.A. (Baddest Of Them All),Warner Records / X One House,UK,39
4,5,Onerepublic,I Ain'T Worried,Interscope,UK,39
...,...,...,...,...,...,...
95,96,Belters Only,I Will Survive,Polydor,UK,30
96,97,Giveon,Lost Me,Black Butter/Epic,UK,30
97,98,Tom Odell,Another Love,Columbia,UK,30
98,99,George Ezra,Anyone For You,Columbia,UK,30


In [ ]:
df['artist'].value_counts()

Harry Styles                 30
Beyonce                      28
Ed Sheeran                   26
Sam Fender                   20
Lizzo                        20
                             ..
Stephen Sanchez               1
Mk & Burns Ft Teddy Swims     1
Lil Baby                      1
Katie Gregson-Macleod         1
Belters Only                  1
Name: artist, Length: 150, dtype: int64